# INSTALLING AND IMPORTING DEPENDENCIES 

In [5]:
import mediapipe as mp
import cv2

In [6]:
!pip install mediapipe opencv-python pandas scikit-learn

In [4]:
pip install -U opencv-python

Note: you may need to restart the kernel to use updated packages.


In [8]:
#drawing utilities 
mp_drawing= mp.solutions.drawing_utils 
mp_holistic=mp.solutions.holistic 

Make some detections 

In [5]:
cap= cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened(): #when camera is opened 
        ret, frame = cap.read()
        
        # Recoloring 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #performance tuning -prevent copying the image data 
        image.flags.writeable=False
               
        
        # Make Detections -using process method by passing our image 
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [6]:
results.face_landmarks.landmark[0].visibility

0.0

In [6]:
import csv
import os 
import numpy as np

In [7]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords


501

In [8]:
landmarks = ['class']
for val in range(1, num_coords+1):
    #coordiantes of each value 
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [9]:
landmarks[-4]

'x501'

In [10]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [11]:
class_name="peace"

In [13]:
cap= cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened(): #when camera is opened 
        ret, frame = cap.read()
        
        # Recoloring 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #performance tuning -prevent copying the image data 
        image.flags.writeable=False
               
        
        # Make Detections -using process method by passing our image 
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)
        
        # Export coordinates
        try:
            # Extract Pose landmarks into numpy array 
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks 
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [19]:
pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

In [20]:
face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

In [21]:
face_row

[0.5455479025840759,
 0.48579269647598267,
 -0.02777036279439926,
 0.0,
 0.548555850982666,
 0.44392770528793335,
 -0.047140900045633316,
 0.0,
 0.5458514094352722,
 0.4586978554725647,
 -0.026399241760373116,
 0.0,
 0.5388519763946533,
 0.4045426845550537,
 -0.034373678267002106,
 0.0,
 0.548737645149231,
 0.430753618478775,
 -0.04952443018555641,
 0.0,
 0.5477374792098999,
 0.41478845477104187,
 -0.045129843056201935,
 0.0,
 0.5442377924919128,
 0.3781874477863312,
 -0.01869955100119114,
 0.0,
 0.47453561425209045,
 0.37909942865371704,
 0.006546629127115011,
 0.0,
 0.5432430505752563,
 0.3478816747665405,
 -0.010355288162827492,
 0.0,
 0.5434598326683044,
 0.3304152488708496,
 -0.01086814422160387,
 0.0,
 0.5416945219039917,
 0.26764607429504395,
 0.002413329901173711,
 0.0,
 0.5453597903251648,
 0.491824209690094,
 -0.026750290766358376,
 0.0,
 0.5447921752929688,
 0.496372252702713,
 -0.024196652695536613,
 0.0,
 0.5440900921821594,
 0.4980195164680481,
 -0.020626548677682877,
 0.

In [22]:
row=pose_row+ face_row

In [83]:
row

[0.621487557888031,
 0.5565140843391418,
 -1.6312015056610107,
 0.997755765914917,
 0.6553926467895508,
 0.48154163360595703,
 -1.517991304397583,
 0.9976712465286255,
 0.6735597848892212,
 0.48814186453819275,
 -1.5185201168060303,
 0.9975346922874451,
 0.6917359232902527,
 0.4948701560497284,
 -1.5186665058135986,
 0.996842086315155,
 0.5961682200431824,
 0.460355281829834,
 -1.5631704330444336,
 0.9981452822685242,
 0.5710746049880981,
 0.4535154104232788,
 -1.5633283853530884,
 0.9984222054481506,
 0.5438294410705566,
 0.4483473598957062,
 -1.5640814304351807,
 0.9984835982322693,
 0.6917549967765808,
 0.5265098810195923,
 -0.8382220268249512,
 0.9977603554725647,
 0.4773132801055908,
 0.48336267471313477,
 -1.0362807512283325,
 0.9990686774253845,
 0.6351112127304077,
 0.648389458656311,
 -1.36765456199646,
 0.9970389604568481,
 0.5651527643203735,
 0.6370660662651062,
 -1.426216959953308,
 0.9982883930206299,
 0.7979395389556885,
 0.8584751486778259,
 -0.42798081040382385,
 0.940

# IMPORTING DEPENDENCIES 

In [23]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [24]:
df= pd.read_csv("C://Users//HP//coords.csv")

In [25]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,happy,0.584228,0.591393,-1.129543,0.995848,0.613516,0.547850,-1.032183,0.994948,0.629144,...,0.020238,0.0,0.654858,0.554063,0.040141,0.0,0.661905,0.549747,0.041304,0.0
1,happy,0.584319,0.591129,-1.184704,0.996130,0.613891,0.544840,-1.104061,0.995113,0.629551,...,0.018159,0.0,0.655246,0.543894,0.036250,0.0,0.662188,0.539839,0.037229,0.0
2,happy,0.581778,0.587915,-1.228768,0.996246,0.612038,0.540542,-1.139270,0.995129,0.628209,...,0.018088,0.0,0.648487,0.536975,0.037834,0.0,0.655253,0.532892,0.038949,0.0
3,happy,0.567877,0.585229,-1.223480,0.996571,0.602011,0.536267,-1.136238,0.995508,0.619687,...,0.018734,0.0,0.636816,0.537159,0.039527,0.0,0.643771,0.532059,0.040790,0.0
4,happy,0.564050,0.585024,-1.245904,0.996775,0.596934,0.534214,-1.158699,0.995683,0.615104,...,0.018541,0.0,0.629691,0.540723,0.039618,0.0,0.636602,0.535062,0.040879,0.0


In [41]:
df[df['class']=='sad']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501


In [27]:
X= df.drop('class',axis=1)

In [28]:
y=df['class']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [30]:
y_test

564      peace
288        sad
298        sad
339        sad
557      peace
        ...   
93       happy
384    victory
543      peace
504      peace
278        sad
Name: class, Length: 183, dtype: object

### Training machine learning classification model 

In [31]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier 
from sklearn.ensemble import RandomForestClassifier ,GradientBoostingClassifier 

In [32]:
pipelines= {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [33]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb'])

In [34]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [35]:
fit_models={}
for algo, pipeline in pipelines.items():
    model_fit =pipeline.fit(X_train, y_train)
    fit_models[algo]= model_fit 

C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [97]:
# fit_models['rf'].predict(X_test)

### Evaluate model

In [36]:
from sklearn.metrics import accuracy_score 
import pickle 

In [37]:
for algo ,model in fit_models.items():
    ypred= model.predict(X_test)
    print(algo, accuracy_score(ypred,y_test))

lr 1.0
rc 1.0
rf 1.0
gb 0.994535519125683


In [38]:
fit_models['lr'].predict(X_test)

array(['peace', 'sad', 'sad', 'sad', 'peace', 'happy', 'peace', 'happy',
       'peace', 'peace', 'sad', 'happy', 'sad', 'sad', 'sad', 'peace',
       'sad', 'sad', 'peace', 'happy', 'peace', 'peace', 'victory',
       'happy', 'peace', 'sad', 'happy', 'happy', 'happy', 'victory',
       'sad', 'happy', 'sad', 'sad', 'peace', 'victory', 'happy', 'sad',
       'happy', 'peace', 'victory', 'happy', 'happy', 'happy', 'peace',
       'peace', 'peace', 'victory', 'sad', 'sad', 'sad', 'peace',
       'victory', 'happy', 'victory', 'peace', 'peace', 'peace', 'sad',
       'victory', 'happy', 'happy', 'happy', 'sad', 'peace', 'happy',
       'happy', 'victory', 'sad', 'happy', 'sad', 'peace', 'victory',
       'sad', 'happy', 'victory', 'happy', 'happy', 'peace', 'sad',
       'peace', 'happy', 'sad', 'sad', 'victory', 'happy', 'peace', 'sad',
       'sad', 'victory', 'sad', 'sad', 'victory', 'sad', 'sad', 'peace',
       'happy', 'victory', 'peace', 'peace', 'sad', 'sad', 'happy',
       'vic

In [39]:
y_test

564      peace
288        sad
298        sad
339        sad
557      peace
        ...   
93       happy
384    victory
543      peace
504      peace
278        sad
Name: class, Length: 183, dtype: object

save model 

In [40]:
#wb is write as binary file 
with open('body_language.pkl','wb') as f :
    pickle.dump(fit_models['lr'], f)

### Make detections on the model 

In [41]:
# read as binary file 
with open('body_language.pkl','rb') as f:
    model= pickle.load(f)

In [42]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [14]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#             # Get status box
#             cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
#             # Display Class
#             cv2.putText(image, 'CLASS'
#                         , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#             cv2.putText(image, body_language_class.split(' ')[0]
#                         , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
#             # Display Probability
#             cv2.putText(image, 'PROB'
#                         , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#             cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
#                         , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
    